<a href="https://colab.research.google.com/github/Coreight98/Paran_ajou_Project_Anything/blob/main/CNN_Furniture_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from sklearn .model_selection import train_test_split
from torchvision.models import resnet50
import time



In [ ]:
train_location = './img/train/'
test_location = './img/val/'

#합성곱 신경망(Convolution Neural Network) 정의하기
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(179776, 17000)
#         self.fc2 = nn.Linear(1700, 170)
#         self.fc3 = nn.Linear(170, 10)
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 179776)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        self.layer1 = torch.nn.Sequential(,
            torch.nn.Conv2d(1, 8, 4),#패딩추가
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(8, 16, 4),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 
        self.fc = torch.nn.Linear(44944, 5, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

class FurnitureDataset():
    def __init__(self, image, class_to_int, mode, transforms):
        super().__init__()
        self.image = image
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms

    def __getitem__(self, index):
        image_name = self.image[index]
        image = Image.open(train_location + image_name)
        image = image.resize((224,224))

        if self.mode == 'train' :
            #레이블 입력
            label = self.class_to_int[image_name.split('.')[0]]
            label = torch.tensor(label,dtype=torch.long)
            #이미지 변형 적용
            image = self.transforms(image)
            return image, label
        elif self.mode == 'test':
            #이미지 변형 적용용
            image = self.transforms(image)
            return image

    def __len__(self):
        return len(self.image)


if __name__ == '__main__':
    #데이터 셋 불러오고 정규화
    transform = transforms.Compose(
        [transforms.ToTensor(),
         #transforms.Grayscale(num_output_channels=1),
          transforms.Normalize((0.5),(0.5))]
    )

    train_images = os.listdir(train_location)
    test_images = os.listdir(test_location)
    print("--Train Images--")
    print("Num of All Images: ", len(train_images))

    beds_list = [img for img in train_images if img.split(".")[0] == "bed"]
    chairs_list = [img for img in train_images if img.split(".")[0] == "chair"]
    sofas_list = [img for img in train_images if img.split(".")[0] == "sofa"]
    swivelchair_list = [img for img in train_images if img.split(".")[0] == "swivelchair"]
    tables_list = [img for img in train_images if img.split(".")[0] == "table"]
    print("Num of Beds Images: ", len(beds_list))
    print("Num of chairs Images: ", len(chairs_list))
    print("Num of sofas Images: ", len(sofas_list))
    print("Num of swivelchairs Images: ", len(swivelchair_list))
    print("Num of tables Images: ", len(tables_list))
    print("--Test Images--")
    print("Num of Images: ", len(test_images))

    # 레이블 구분
    class_to_int = {"bed": 0, "chair": 1, "sofa": 2, "swivelchair": 3, "table": 4}
    int_to_class = {0: "bed", 1: "chair", 2: "sofa", 3: "swivelchair", 4: "table"}
    #classes = {'bed', 'chair', 'sofa', 'swivelchair', 'table'}

    # 데이터 셋 생성
    train_dataset = FurnitureDataset(train_images, class_to_int, mode='train', transforms=transform)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)

    test_dataset = FurnitureDataset(test_images, class_to_int, mode='test', transforms=transform)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)

    # net = Net()
    net = CNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    device = 'cuda'
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print (device,"로 학습")

    # 신경망 학습하기
    for epoch in range(2):
        running_loss = 0.0
        for i, data in enumerate(trainloader,0):
            inputs, labels = data  # data입력
            optimizer.zero_grad()  # gradient 매개변수를 0으로 만듬
            # 순전파 + 역전파 + 최적화
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # 통계출력
            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    print('Finished Training')
    PATH = './cifar_net.pth'
    torch.save(net.state_dict(), PATH)

--Train Images--
Num of All Images:  1722
Num of Beds Images:  900
Num of chairs Images:  634
Num of sofas Images:  60
Num of swivelchairs Images:  89
Num of tables Images:  39
--Test Images--
Num of Images:  423
cuda 로 학습


RuntimeError: ignored